In [1]:
import pandas as pd
from BOCC import BOCC, load_clusters
import matplotlib.pyplot as plt
import networkx as nx
import os
import obonet
import pandas as pd
import typing
import numpy as np

/Users/michael/miniconda3/envs/bocc/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
url = 'https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.obo'
graph = obonet.read_obo(url)
print(graph.nodes(data=True)['HP:0001250']['name'])

Seizure


In [3]:
G21 = nx.read_edgelist('../Edgelists/String_HPO_2021.phenotypic_branch.edgelist.txt')
G22 = nx.read_edgelist('../Edgelists/String_HPO_2022.phenotypic_branch.edgelist.txt')

# Load the information about predicted 2021 clusters
predicted_df = pd.read_csv('../XGBoost2021ClusterRankings.csv')
print(predicted_df.columns)
print(predicted_df.shape)

# Load all the clusters
coms_dict = {}
for dir in os.listdir('../SubComs'):
    if dir in ['2021','2022']:
        for f in os.listdir('../SubComs/{}'.format(dir)):
            if 'paris' in f:
                coms_dict[f.replace('.coms.txt','')] = load_clusters('../SubComs/{}/{}'.format(dir,f))

Index(['clusterID', 'estimated_snowballing_pvalue',
       'max_norm_cell_type_specificity', 'max_norm_disease_specificity',
       'max_plof', 'mean_plof', 'median_plof', 'num_cell_type_string',
       'num_disease_string', 'num_of_diseases', 'num_sig_go_enrichment_terms',
       'std_plof', 'sum_plof', 'avg_embeddedness', 'avg_internal_degree',
       'cluster_size', 'conductance', 'cut_ratio', 'edges_inside', 'expansion',
       'gene_ratio', 'hub_dominance', 'internal_edge_density',
       'newman_girvan_modularity', 'normalized_cut', 'significance',
       'surprise', 'triangle_participation_ratio'],
      dtype='object')
(3656, 28)


create a dictionary of com_id to com_predicted score

In [4]:
# create a percentile - the portion of clusters this a cluster is predicted to be MORE "significant" than, e.i. lower p-value
predicted_df['percentile'] = [
    sum(predicted_df['estimated_snowballing_pvalue'] <= x)/predicted_df.shape[0] 
    for x in predicted_df['estimated_snowballing_pvalue']]

com_score_map = {r[['clusterID']].values[0]:r[['estimated_snowballing_pvalue']].values[0] for i,r in predicted_df.iterrows()}
com_percentile_map = {r[['clusterID']].values[0]:round(r[['percentile']].values[0], 2) for i,r in predicted_df.iterrows()}
# print(com_score_map)

# MyGene 2

In [5]:
# Load the MyGene2 Familes
mg2_df = pd.read_csv('../Resources/my_gene_2_variantes_by_family_tables.csv')
print(mg2_df.columns)
print(mg2_df.shape)

# create a dict with patient keys and lists of all HPOs and genes within them
# id: [gene1,hpo1,gene2,hpo2...]
patient_map = {}
# add all the genes
for i,r in mg2_df.groupby('family_id')['gene'].agg(['unique'])['unique'].reset_index().iterrows():
    patient_map[r['family_id']] = list(r['unique'])
# add all the HPOs
for i,r in mg2_df.groupby('family_id')['phenotype_hpo_id'].agg(['unique'])['unique'].reset_index().iterrows():
    if r['family_id'] in patient_map:
            patient_map[r['family_id']] = patient_map[r['family_id']] + list(r['unique'])
print(len(patient_map))
assert(len(patient_map) == len(set(mg2_df['family_id'])))


Index(['Unnamed: 0', 'gene', 'variant_id', 'family_id', 'family_url',
       'inheritance', 'phenotype_name', 'phenotype_hpo_id',
       'confidence_in_pathogenicity'],
      dtype='object')
(21455, 9)
912


In [6]:
filtered_patient_map = {}
for key in patient_map.keys():
    if sum(['HP:' not in x for x in patient_map[key]]) > 0:
        filtered_patient_map[key] = patient_map[key]
print(len(patient_map))
print(len(filtered_patient_map))

912
912


create all possible pairs for each patient

In [7]:
all_patient_pairs = {}
for key in filtered_patient_map.keys():
    genes = [x for x in patient_map[key] if 'HP:' not in x]
    hpos = [x for x in patient_map[key] if 'HP:' in x]
    all_patient_pairs[key] = [[g,h] for g in genes for h in hpos]

How many of these pateints do not have any edges already in the 2019 graph?

In [8]:
solved_patients = []
for key in all_patient_pairs.keys():
    for pair in all_patient_pairs[key]:
        if G21.has_edge(pair[0],pair[1]) or G21.has_edge(pair[1],pair[0]):
            solved_patients.append(key)
            break
unsolved_patients = [x for x in all_patient_pairs.keys() if x not in solved_patients]
print('Number of "unsolved" MyGene2 patients',len(unsolved_patients))

unsolved_patients_map = {id:filtered_patient_map[id] for id in unsolved_patients}
unsolved_patients_pairs = {id:all_patient_pairs[id] for id in unsolved_patients}

Number of "unsolved" MyGene2 patients 115


## How many of these pateints have one of their edges discovered in a cluster?

In [9]:
# {101:['paris.censa.2021:1:SEMA4A,HP:0000128']} patient_id:list of ids of com this patient was solved by:gene,HPO
newly_solved_patients = {}
mg2_edges = []
for pid in unsolved_patients_pairs:
    # is this pid's pairs in any of the coms?
    for pair in unsolved_patients_pairs[pid]:
        # for each community sets keys are in formatted like 'paris.cesna.2021'
        for k in coms_dict.keys():
            if '2021' not in k:
                continue
            for com in coms_dict[k]:
                if pair[0] in com.members and pair[1] in com.members:
                    cluster_uid = '{cluster_set}:{cluster_id}'.format(cluster_set=k, cluster_id=str(com.name))
                    res_string = '{pid} & {cluster_set}:{cluster_id} & {node1} & {node2} & {hpo_name} & {rank} \\\\'.format(cluster_set=k,
                                                                                                          cluster_id=str(com.name),
                                                                                                          node1=pair[0],
                                                                                                          node2=pair[1],
                                                                                                          hpo_name=graph.nodes(data=True)[pair[1]]['name'],
                                                                                                                  rank=com_percentile_map[cluster_uid],
                                                                                                                  pid=pid)
                    print(res_string)
                    mg2_edges.append([cluster_uid, pair[0],pair[1]])
                    if pid in newly_solved_patients:
                        newly_solved_patients[pid].append(res_string)
                    else:
                        newly_solved_patients[pid] = [res_string]
                    

print('Number of patients solved by one of these communities', len(newly_solved_patients)) 


1292 & paris.walktrap.2021:1321 & TNNT3 & HP:0002804 & Arthrogryposis multiplex congenita & 0.27 \\
1292 & paris.greedy.2021:968 & TNNT3 & HP:0002804 & Arthrogryposis multiplex congenita & 0.27 \\
1635 & paris.cesna.2021:22018 & SPTBN5 & HP:0001250 & Seizure & 0.37 \\
1922 & paris.cesna.2021:15553 & FAT2 & HP:0001310 & Dysmetria & 0.33 \\
1930 & paris.walktrap.2021:84 & NBEA & HP:0001824 & Weight loss & 0.91 \\
1932 & paris.cesna.2021:9175 & HUWE1 & HP:0000739 & Anxiety & 0.15 \\
1943 & paris.cesna.2021:3199 & SHANK2 & HP:0011968 & Feeding difficulties & 0.19 \\
1948 & paris.cesna.2021:9175 & CHD2 & HP:0001250 & Seizure & 0.15 \\
1950 & paris.cesna.2021:14105 & SATB2 & HP:0001999 & Abnormal facial shape & 0.08 \\
1950 & paris.cesna.2021:5816 & SATB2 & HP:0000202 & Oral cleft & 0.34 \\
2197 & paris.cesna.2021:6379 & C17orf62 & HP:0006532 & Recurrent pneumonia & 0.1 \\
2197 & paris.cesna.2021:6379 & C17orf62 & HP:0001744 & Splenomegaly & 0.1 \\
2234 & paris.cesna.2021:24074 & DROSHA & HP

## 2022 MyGene2 Results

In [10]:
solved_patients_22 = []
for key in all_patient_pairs.keys():
    for pair in all_patient_pairs[key]:
        if G22.has_edge(pair[0],pair[1]) or G22.has_edge(pair[1],pair[0]):
            solved_patients_22.append(key)
            break
unsolved_patients_22 = [x for x in all_patient_pairs.keys() if x not in solved_patients_22]
print('Number of "unsolved" MyGene2 patients',len(unsolved_patients_22))

unsolved_patients_map_22 = {id:filtered_patient_map[id] for id in unsolved_patients_22}
unsolved_patients_pairs_22 = {id:all_patient_pairs[id] for id in unsolved_patients_22}


Number of "unsolved" MyGene2 patients 112


In [11]:
predicted_df_22 = pd.read_csv('../xgb_cluster_rankings_2022.tsv',sep='\t')
predicted_df_22['percentile'] = [
    sum(predicted_df_22['estimated_snowballing_pvalue'] <= x)/predicted_df_22.shape[0] 
    for x in predicted_df_22['estimated_snowballing_pvalue']]

com_score_map_22 = {r[['clusterID']].values[0]:r[['estimated_snowballing_pvalue']].values[0] for i,r in predicted_df_22.iterrows()}
com_percentile_map_22 = {r[['clusterID']].values[0]:round(r[['percentile']].values[0], 2) for i,r in predicted_df_22.iterrows()}



# {101:['paris.censa.2021:1:SEMA4A,HP:0000128']} patient_id:list of ids of com this patient was solved by:gene,HPO
newly_solved_patients_22 = {}
mg2_edges = []
for pid in unsolved_patients_pairs_22:
    # is this pid's pairs in any of the coms?
    for pair in unsolved_patients_pairs_22[pid]:
        # for each community sets keys are in formatted like 'paris.cesna.2021'
        for k in coms_dict.keys():
            if '2022' not in k:
                continue
            for com in coms_dict[k]:
                if pair[0] in com.members and pair[1] in com.members:
                    cluster_uid = '{cluster_set}:{cluster_id}'.format(cluster_set=k, cluster_id=str(com.name))
                    res_string = '{pid} & {cluster_set}:{cluster_id} & {node1} & {node2} & {hpo_name} {rank} \\\\'.format(cluster_set=k,
                                                                                                          cluster_id=str(com.name),
                                                                                                          node1=pair[0],
                                                                                                          node2=pair[1],
                                                                                                          hpo_name=graph.nodes(data=True)[pair[1]]['name'],
                                                                                                                  rank='',
#                                                                                                                   rank='TBD',
                                                                                                                  pid=pid)
                    print(res_string)
                    mg2_edges.append([cluster_uid, pair[0],pair[1]])
                    if pid in newly_solved_patients_22:
                        newly_solved_patients_22[pid].append(res_string)
                    else:
                        newly_solved_patients_22[pid] = [res_string]
                    

print('Number of patients solved by one of these communities', len(set(newly_solved_patients_22))) 

1292 & paris.walktrap.2022:1369 & TNNT3 & HP:0002804 & Arthrogryposis multiplex congenita  \\
151 & paris.infomap.2022:973 & PCOLCE2 & HP:0000545 & Myopia  \\
1930 & paris.cesna.2022:5864 & NBEA & HP:0001250 & Seizure  \\
2197 & paris.cesna.2022:17456 & C17orf62 & HP:0001945 & Fever  \\
2197 & paris.cesna.2022:17456 & C17orf62 & HP:0006532 & Recurrent pneumonia  \\
2197 & paris.cesna.2022:17456 & C17orf62 & HP:0001744 & Splenomegaly  \\
2197 & paris.cesna.2022:17456 & C17orf62 & HP:0002716 & Lymphadenopathy  \\
2234 & paris.cesna.2022:15933 & DROSHA & HP:0000252 & Microcephaly  \\
2525 & paris.cesna.2022:5117 & FMN1 & HP:0004322 & Short stature  \\
2584 & paris.cesna.2022:4308 & PYROXD1 & HP:0003198 & Myopathy  \\
2649 & paris.walktrap.2022:1626 & BCL6B & HP:0001596 & Alopecia  \\
27 & paris.cesna.2022:3984 & SCN2A & HP:0001250 & Seizure  \\
2748 & paris.cesna.2022:18491 & DROSHA & HP:0002015 & Dysphagia  \\
347 & paris.cesna.2022:11079 & GUCY1A3 & HP:0000822 & Hypertension  \\
8 & par

In [12]:
# number of unsolved cases in 2021 and 2022
print('Number of "unsolved" MyGene2 patients 2021',len(unsolved_patients))
print('Number of "unsolved" MyGene2 patients 2022',len(unsolved_patients_22))
no_longer_undiagnosed = [x for x in unsolved_patients if x not in unsolved_patients_22]
new_undiagnosed = [x for x in unsolved_patients_22 if x not in unsolved_patients]
print(no_longer_undiagnosed)
print(new_undiagnosed)
# print(unsolved_patients)
# print(unsolved_patients_22)
print(all_patient_pairs['1930'])
for d in no_longer_undiagnosed:
    print(d)
    for p in all_patient_pairs[d]:
        if G22.has_edge(p[0],p[1]):
            print(d, p)


Number of "unsolved" MyGene2 patients 2021 115
Number of "unsolved" MyGene2 patients 2022 112
['124', '2232', '2240', '2242', '2830', '2832']
['21', '22', '2611']
[['SSPO', 'HP:0001251'], ['SSPO', 'HP:0001250'], ['SSPO', 'HP:0001289'], ['SSPO', 'HP:0001733'], ['SSPO', 'HP:0100279'], ['SSPO', 'HP:0002014'], ['SSPO', 'HP:0010543'], ['SSPO', 'HP:0002066'], ['SSPO', 'HP:0002186'], ['SSPO', 'HP:0000496'], ['SSPO', 'HP:0002574'], ['SSPO', 'HP:0001824'], ['SSPO', 'HP:0001336'], ['SSPO', 'HP:0001337'], ['NBEA', 'HP:0001251'], ['NBEA', 'HP:0001250'], ['NBEA', 'HP:0001289'], ['NBEA', 'HP:0001733'], ['NBEA', 'HP:0100279'], ['NBEA', 'HP:0002014'], ['NBEA', 'HP:0010543'], ['NBEA', 'HP:0002066'], ['NBEA', 'HP:0002186'], ['NBEA', 'HP:0000496'], ['NBEA', 'HP:0002574'], ['NBEA', 'HP:0001824'], ['NBEA', 'HP:0001336'], ['NBEA', 'HP:0001337']]
124
124 ['TCF7L2', 'HP:0001252']
124 ['TCF7L2', 'HP:0001250']
2232
2232 ['CHD5', 'HP:0001250']
2240
2240 ['TNPO2', 'HP:0000252']
2240 ['TNPO2', 'HP:0001252']
2240 [

# Mammalian Phenotype Ontology

In [13]:
# Load the infered edges
hpo_from_mpo = pd.read_csv('../Resources/hpo_to_gene_derived_by_mpo.edgelist.2021.unique.txt',sep='\t',header=None)
hpo_from_mpo.columns = ['HPO','Gene','Name']
print(hpo_from_mpo)

           HPO    Gene                                  Name
0   HP:0000086  FANCD2                        Ectopic kidney
1   HP:0000093    G6PC                           Proteinuria
2   HP:0000097    G6PC    Focal segmental glomerulosclerosis
3   HP:0000105    G6PC                       Enlarged kidney
4   HP:0000238    MSH2                         Hydrocephalus
..         ...     ...                                   ...
80  HP:0011463     XPC                       Childhood onset
81  HP:0012213    G6PC  Decreased glomerular filtration rate
82  HP:0020073    MSH2                  Hypopigmented macule
83  HP:0030052    MSH2                    Inguinal freckling
84  HP:0100697    MSH2                     Neurofibrosarcoma

[85 rows x 3 columns]


In [14]:
newly_solved_patients = {}
mpo_pairs = [ [hpo_from_mpo.iloc[i,:]['Gene'],hpo_from_mpo.iloc[i,:]['HPO'] ]for i in range(hpo_from_mpo.shape[0])]
mpo_discovering_coms = []
for pair in mpo_pairs:
    # for each community sets keys are in formatted like 'paris.cesna.2021'
    for k in coms_dict.keys():
        if '2021' not in k:
            continue
        for com in coms_dict[k]:
            if pair[0] in com.members and pair[1] in com.members:
                cluster_uid = '{cluster_set}:{cluster_id}'.format(cluster_set=k, cluster_id=str(com.name))
                mpo_discovering_coms.append(cluster_uid)
                res_string = '{cluster_set}:{cluster_id} & {node1} & {node2} & {hpo_name} & {rank} \\\\'.format(cluster_set=k, 
                                                                                                                cluster_id=str(com.name), 
                                                                                                                node1=pair[0], 
                                                                                                                node2=pair[1], 
                                                                                                                hpo_name=graph.nodes(data=True)[pair[1]]['name'],
                                                                                                                rank='')
                print(res_string)

paris.infomap.2021:1828 & FANCD2 & HP:0000086 & Ectopic kidney &  \\
paris.cesna.2021:22354 & FANCD2 & HP:0000086 & Ectopic kidney &  \\
paris.walktrap.2021:1309 & ITGB3 & HP:0000979 & Purpura &  \\
paris.greedy.2021:956 & ITGB3 & HP:0000979 & Purpura &  \\
paris.cesna.2021:14597 & TDGF1 & HP:0001263 & Global developmental delay &  \\
paris.walktrap.2021:1309 & ITGB3 & HP:0001892 & Abnormal bleeding &  \\
paris.greedy.2021:956 & ITGB3 & HP:0001892 & Abnormal bleeding &  \\
paris.infomap.2021:1697 & ITGB3 & HP:0004813 & Post-transfusion thrombocytopenia &  \\


In [15]:
def log_2_ratio(o: int, e: typing.List[int]) -> float:
    """
    param o: observed number of rediscoveries
    param e: list of expected number of rediscoveries based on the null model
    """
    return float(np.log2(o / np.median(np.array(e))))

def empircal_p(o: int, e: typing.List[int]) -> float:
    """
    param o: observed number of rediscoveries
    param e: list of expected number of rediscoveries based on the null model
    """
    return float(1 - (sum([ o > x for x in e]) / len(e)))

mouse_snow_dfs = []
mouse_files = ['snowball.paris.greedy.String_HPO_2021.mouse.phenotypic_branch.tsv',
              'snowball.paris.walktrap.String_HPO_2021.mouse.phenotypic_branch.tsv',
              'snowball.paris.infomap.String_HPO_2021.mouse.phenotypic_branch.tsv',
              'snowball.paris.cesna.String_HPO_2021.mouse.phenotypic_branch.tsv']

for i,file in enumerate(mouse_files):
    tmp_df = pd.read_csv('../MouseSnowballResults/'+file,sep='\t')
    mouse_snow_dfs.append(tmp_df)


mouse_l2r_p = {'algorithm':[],'cluster_id':[],'log2ratio':[],'p-value':[]}
algos = ['paris-greedy','paris-walktrap','paris-infomap','paris-cesna']
# print stats for all DFs
# calc log2ratio and p value for each cluster in each algo category. Save it all in one dataframe
for df,algo in zip(mouse_snow_dfs, algos):
    for cid in df['com_id'].unique():
        sub = df[df['com_id'] == cid]
        observed = list(sub['com_score'])[0]
        expected = list(sub['replicate_score'])
        l2r = log_2_ratio(observed,expected)
        p = empircal_p(observed,expected)
        mouse_l2r_p['algorithm'].append(algo)
        mouse_l2r_p['cluster_id'].append(cid)
        mouse_l2r_p['log2ratio'].append(l2r)
        mouse_l2r_p['p-value'].append(p)
mouse_l2r_p_df = pd.DataFrame(mouse_l2r_p)

print(mouse_l2r_p_df.columns)
print(mouse_l2r_p_df.shape)

/var/folders/ty/694s8g7n1cz_l5s3prx55jzr0000gn/T/ipykernel_2435/2035549221.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return float(np.log2(o / np.median(np.array(e))))
/var/folders/ty/694s8g7n1cz_l5s3prx55jzr0000gn/T/ipykernel_2435/2035549221.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  return float(np.log2(o / np.median(np.array(e))))


Index(['algorithm', 'cluster_id', 'log2ratio', 'p-value'], dtype='object')
(32767, 4)


In [16]:
# print(mouse_l2r_p_df)
mouse_l2r_p_df['algorithm'] = [x.replace('-','.') for x in mouse_l2r_p_df['algorithm']]
mouse_l2r_p_df['uid'] = mouse_l2r_p_df['algorithm'] + '.2021:' + mouse_l2r_p_df['cluster_id'].astype(str)
print(mpo_discovering_coms)
print(mouse_l2r_p_df['uid'])
for uid in mpo_discovering_coms:
    print(mouse_l2r_p_df[mouse_l2r_p_df['uid'] == uid])

['paris.infomap.2021:1828', 'paris.cesna.2021:22354', 'paris.walktrap.2021:1309', 'paris.greedy.2021:956', 'paris.cesna.2021:14597', 'paris.walktrap.2021:1309', 'paris.greedy.2021:956', 'paris.infomap.2021:1697']
0           paris.greedy.2021:0
1           paris.greedy.2021:1
2           paris.greedy.2021:2
3           paris.greedy.2021:3
4           paris.greedy.2021:4
                  ...          
32762    paris.cesna.2021:26520
32763    paris.cesna.2021:26521
32764    paris.cesna.2021:26522
32765    paris.cesna.2021:26523
32766    paris.cesna.2021:26524
Name: uid, Length: 32767, dtype: object
          algorithm  cluster_id  log2ratio  p-value                      uid
5347  paris.infomap        1828        inf      0.0  paris.infomap.2021:1828
         algorithm  cluster_id  log2ratio  p-value                     uid
28596  paris.cesna       22354        inf      0.0  paris.cesna.2021:22354
           algorithm  cluster_id  log2ratio  p-value                       uid
2527  paris.

# MyGene2 Result Shortest Paths
How are the new pairs connected to each other in their clusters? Do they pass through genes? Through HPOs? If HPOs it could mean it is just a HPO specificity problem.

In [17]:
# UID, HPO, GENE
mg2_edges

path_res = {'algo':[],'com':[],'pair':[],'num_genes':[]}
num_paths = []
num_hpos_in_path = []
for case in mg2_edges:
    key = case[0].split(':')[0]
    com_id = case[0].split(':')[1]
    # find the cluster from the list of clusters
    com = [c for c in coms_dict[key] if str(c.name) == com_id ][0]
    # get the shortest path(s)
    g = G21.subgraph(com.members)
    paths = list(nx.all_shortest_paths(g, case[1],case[2]))
    num_paths.append(len(paths))
    for path in paths:
#         print(path)
        num_hpos_in_path.append(sum(['HP:' in n for n in path[1:-1]]))
        if num_hpos_in_path[-1] > 0:
            print('THIS ONE',path)
        print('{algo}:{com} {path}'.format(algo=key,com=com_id,path=str(path[1:-1])))

# number of paths per pair
print('Num paths:', num_paths)

# how many have a shortest path with an HPO in it?
print('Num HPOs in paths',num_hpos_in_path)
print(sum(num_hpos_in_path))

paris.walktrap.2022:1369 ['TNNI2']
paris.walktrap.2022:1369 ['MYH3']
paris.walktrap.2022:1369 ['PIEZO2']
paris.walktrap.2022:1369 ['TPM2']
paris.walktrap.2022:1369 ['CHRNG']
paris.walktrap.2022:1369 ['ECEL1']
paris.infomap.2022:973 ['COL4A1']
paris.infomap.2022:973 ['PLOD3']
paris.infomap.2022:973 ['ADAMTS2']
paris.infomap.2022:973 ['COL9A3']
paris.infomap.2022:973 ['COL11A1']
paris.infomap.2022:973 ['COL4A4']
paris.infomap.2022:973 ['COL4A5']
paris.infomap.2022:973 ['COL9A2']
paris.infomap.2022:973 ['COL9A1']
paris.cesna.2022:5864 ['PTEN']
paris.cesna.2022:5864 ['STXBP1']
paris.cesna.2022:5864 ['DLL1']
paris.cesna.2022:17456 ['NCF4']
paris.cesna.2022:17456 ['NCF1']
paris.cesna.2022:17456 ['CYBB']
paris.cesna.2022:17456 ['CYBA']
paris.cesna.2022:17456 ['NCF2']
paris.cesna.2022:17456 ['NCF1']
paris.cesna.2022:17456 ['CYBB']
paris.cesna.2022:17456 ['CYBA']
paris.cesna.2022:17456 ['NCF2']
paris.cesna.2022:17456 ['NCF4']
paris.cesna.2022:17456 ['NCF1']
paris.cesna.2022:17456 ['CYBB']
paris